In [ ]:
%matplotlib inline

import numpy as np
import scipy.stats as stats

from scipy.special import gammaln
from matplotlib import pyplot as plt

import delfi.generator as dg
import delfi.distribution as dd
from delfi.utils.viz import plot_pdf
import delfi.inference as infer
from delfi.summarystats.Identity import Identity

from snl.util.plot import plot_hist_marginals
from lfimodels.snl_exps.util import  save_results_byname, load_results_byname
from lfimodels.rockpaperscissors.rps_sde import rps_sde

model_id = 'rps'
save_path = 'results/' + model_id



seed = 42

L = 100  # image rows/columns
duration = 100.0
dt = 1.0

nseeds = 10

use_maf = False

In [16]:
# simulation setup
setup_opts = {
    'n_bypass': 0,
    'filter_sizes': [3,3,3,3, 2, 2],
    'n_filters':    (8,8,8,16,32,32),
    'pool_sizes':   [1,3,2,2, 2, 1],
    'n_hiddens': [50, 50],
    'reg_lambda': 0.01,
    'pilot_samples': 0,  # values are already between 0 and 1
    'verbose': True,
    'prior_norm': False,  # turn off till we've fixed the active trainer. good for uniform priors etc.
    'init_norm': False,
    'input_shape': (3,L,L),
    'verbose': True,
}
if use_maf:
    setup_opts.update(dict(density='maf', maf_mode='random', n_mades=5, maf_actfun='tanh', batch_norm=False))
else:
    setup_opts.update(dict(density='mog', n_components=1))                 

run_opts = {
    'n_train': 1000,
    'n_rounds': 10,
    'minibatch': 100,
    'epochs': 2000,
    'moo': 'resample',
    'proposal': 'discrete',
    'n_null': None,
    'train_on_all': True,
    'max_norm': 0.1,
    'val_frac': 0.1,
    'silent_fail': False,
}

In [ ]:
# define a function for showing simulation results as images
def showsim(s_rgb, **kwargs):
    if 'interpolation' not in kwargs.keys():
        kwargs['interpolation'] = 'None'
    s_ryb = s_rgb.copy().reshape(3, L, L)
    s_ryb[0] = np.minimum(1.0, s_ryb[0] + s_ryb[1])
    imobj = plt.imshow(np.moveaxis(s_ryb, 0, -1), **kwargs)
    plt.gca().axis('off')
    return imobj

In [18]:
def init_all(seed, pars_true=None, infobj=infer.SNPEC, svi=False):
    p = dd.Uniform(lower=np.array([-1, -1, -6], dtype=float), 
                   upper=np.array([1, 1, -5], dtype=float), seed=seed)
    n_cores = 8    
    mlist = [rps_sde(dt=dt, duration=duration, L=L, seed=seed + 100 + i) for i in range(n_cores)]
    g = dg.MPGenerator(models=mlist, prior=p, summary=Identity(seed=seed+2), seed=seed+3)
    
    #m = rps_sde(dt=dt, duration=duration, L=L, seed=seed+1)
    #g = dg.Default(model=m, prior=p, summary=Identity(seed=seed+2), seed=seed+3)
    
    if pars_true is None:
        pars_true = g.prior.gen(1).reshape(-1)
    #obs = g.model.gen_single(pars_true)
    obs = g.models[0].gen_single(pars_true)
    obs_stats = g.summary.calc([obs])
    
    if setup_opts['density'] == 'mog':
        setup_opts['svi'] = svi
    
    inf = infobj(generator=g, obs=obs_stats, seed=seed+4, **setup_opts)
    
    return g, pars_true, obs, obs_stats, inf

In [ ]:
pars_true = np.array([-0.5, 0.5, -5.25])

g, pars_true, obs, obs_stats, inf = init_all(seed, pars_true=pars_true)

showsim(obs_stats)

In [ ]:
# print the network structure. "None" indicates the batch dimension
for s in inf.network.layer:
    if s.startswith('mixture'):
        continue
    print('{0}: {1}'.format(s, inf.network.layer[s].output_shape))

In [ ]:
log, trn_data, posteriors = inf.run(**run_opts)

In [ ]:
fig_conv = plt.figure(figsize=(8, 8))
for r in range(run_opts['n_rounds']):
    plt.subplot(run_opts['n_rounds'], 1, r + 1)
    plt.plot(log[r]['loss'],'b',label='training loss')
    plt.plot(log[r]['val_loss_iter'], log[r]['val_loss'], 'm', label='validation loss')
    plt.legend()

In [ ]:
plot_hist_marginals(posteriors[-1].gen(1000), lims=np.vstack((g.prior.lower, g.prior.upper)).T, 
                            gt=pars_true.flatten(), rasterized=False)

In [ ]:
labels_params = ['$\log_{10}(\mu)$', '$\log_{10}(\sigma)$', '$\log_{10}(D)$'] 
# all pairwise marginals of fitted posterior
fig_posterior, _ = plot_pdf(posteriors[-1], lims=[[-1, 1], [-1, 1], [-7, -5]], gt=pars_true.reshape(-1), figsize=(8, 8), resolution=100,
                  labels_params=labels_params, ticks=True)

In [ ]:
fig_nlogptrue = plt.figure()

nsim = (np.arange(run_opts['n_rounds']) + 1.0) * run_opts['n_train']

plt.semilogx(nsim, [-pp.eval(pars_true.reshape(1, -1), log=True) for pp in posteriors],'.-')
plt.ylabel('-log p(true parameters)')
plt.xlabel('Number of simulations (log scale)')

In [ ]:
# draw samples from the prior and posterior
n_samples = 5

g.proposal = None
x_prior = g.gen(n_samples)[1]
x_posterior = np.stack([y[0]['data'] for y in g.model.gen(posteriors[-1].gen(n_samples))], axis=0)

In [ ]:
fig_samples = plt.figure(figsize=(8, 8))
plt.subplot(3, n_samples, 1)
showsim(obs_stats)
plt.title('Observed data')
for i in range(n_samples):
    plt.subplot(3, n_samples, n_samples + i + 1)
    showsim(x_posterior[i])
    if i == 0:
        plt.title('Posterior samples')
    plt.subplot(3, n_samples, 2 * n_samples + i + 1)
    showsim(x_prior[i])
    if i == 0:
        plt.title('Prior samples')

In [ ]:
fig_posterior.savefig('rps_posterior.pdf')
fig_samples.savefig('rps_samples.pdf')
fig_nlogptrue.savefig('rps_nlogptrue.pdf')

!pdftk rps_posterior.pdf rps_samples.pdf rps_nlogptrue.pdf cat output rps.pdf
!rm rps_posterior.pdf
!rm rps_samples.pdf
!rm rps_nlogptrue.pdf

In [5]:
seeds = seed + np.arange(nseeds) * 20

In [ ]:
# run multiple seeds with SNPEC discrete MDN
for s in seeds:
    _, pt, o, os, nextinf = init_all(s)  # samples observed data from the marginal (i.e. with parameters from the prior)
    ll, _, pp = nextinf.run(**run_opts)
    
    exp_id = 'rps_' + setup_opts['density'] + '_' + str(s)
    save_results_byname(logs=ll, posteriors=pp, raw_data=o, pars_true=pt, seed=s,
                        setup_opts=setup_opts, run_opts=run_opts, exp_id=exp_id, path=save_path)

In [20]:
run_opts_SNPEAB = {
    'n_train': 1000,
    'n_rounds': 10,
    'minibatch': 100,
    'epochs': 2000,
    'max_norm': 0.1,
    'val_frac': 0.1,
}

In [15]:
# run multiple seeds with SNPEA MDN
for s in seeds:
    
    _, pt, o, os, nextinf = init_all(s, infobj=infer.CDELFI)  # samples observed data from the marginal (i.e. with parameters from the prior)
    ll, _, pp = nextinf.run(**run_opts_SNPEAB)
    
    exp_id = 'rps_' + setup_opts['density'] + '_' + str(s) + '_A'
    save_results_byname(logs=ll, posteriors=pp, raw_data=o, pars_true=pt, seed=s,
                        setup_opts=setup_opts, run_opts=run_opts, exp_id=exp_id, path=save_path)

round :  1
proposal:  None


Stopping at epoch = 186, training loss = -2.1419299562636676, validation loss = -1.5746778066817586


round :  2
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb83819e710>


Stopping at epoch = 145, training loss = -5.486651302165155, validation loss = -5.4598493679229065


round :  3
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb83818b470>


Stopping at epoch = 146, training loss = -6.4045173557463935, validation loss = -6.033946252650384


round :  4
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb816405358>


Stopping at epoch = 111, training loss = -5.680058208293085, validation loss = -5.6646206863585755


round :  5
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb85c3f78d0>


...

Stopping at epoch = 66, training loss = -6.180287037676935, validation loss = -6.442227345294123


analytic correction for proposal seemingly failed!
round :  7
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb8154720b8>


Stopping at epoch = 154, training loss = -7.597392369097833, validation loss = -7.365776844805842


round :  8
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb814b3ab70>


Stopping at epoch = 120, training loss = -7.343328047579889, validation loss = -7.193799463099603


round :  9
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb849942080>


Stopping at epoch = 122, training loss = -8.85267739850915, validation loss = -8.385101128405696


round :  10
proposal:  <delfi.distribution.Gaussian.Gaussian object at 0x7fb8cd107e10>


Stopping at epoch = 121, training loss = -9.624101014918452, validation loss = -9.184023686236506




In [21]:
# run multiple seeds with SNPEB MDN
for s in seeds:
    
    _, pt, o, os, nextinf = init_all(s, infobj=infer.SNPE, svi=True)  # samples observed data from the marginal (i.e. with parameters from the prior)
    ll, _, pp = nextinf.run(**run_opts_SNPEAB)
    
    exp_id = 'rps_' + setup_opts['density'] + '_' + str(s) + '_B'
    save_results_byname(logs=ll, posteriors=pp, raw_data=o, pars_true=pt, seed=s,
                        setup_opts=setup_opts, run_opts=run_opts, exp_id=exp_id, path=save_path)

Stopping at epoch = 843, training loss = 12.830168310257973, validation loss = 14.497100666981176




Stopping at epoch = 38, training loss = 2.168175440400374, validation loss = -0.24344900623402732




Stopping at epoch = 82, training loss = -8.270109337458452, validation loss = -0.9879007128762896




Stopping at epoch = 46, training loss = -1.2160659280337014, validation loss = -1.1924345958339253




Stopping at epoch = 52, training loss = -0.6293046249324764, validation loss = 0.11690880368036347




...

Stopping at epoch = 32, training loss = -0.4471130164168682, validation loss = -14.597463667963943




Stopping at epoch = 58, training loss = -1.726465355003795, validation loss = 3.9675471464602583




Stopping at epoch = 30, training loss = -1.3153696371268726, validation loss = 0.35220923351855377




Stopping at epoch = 45, training loss = -5.350805871875193, validation loss = -2.4401325367416193




Stopping at epoch = 28, training loss = -0.05931681356670239, validation loss = 8.692485906412285




In [ ]:
#load results
subdirs = os.listdir(save_path)
exp_ids, exp_ids_A = [], []
for d in subdirs:
    if not d.startswith('rps_'):
        continue
    if d.endswith('_A'):
        exp_ids_A.append(d)
    else:
        exp_ids.append(d)
        
posteriors_seeds, pars_true_seeds = [], []
for exp_id in exp_ids:
    pp, load_vars = load_results_byname(exp_id)
    posteriors_seeds.append(pp)
    pars_true_seeds.append(load_vars['pars_true'])
Pmat = np.array([[-pp.eval(pt.reshape(1, -1), log=True)[0] for pp in P] for P, pt in zip (posteriors_seeds, pars_true_seeds)]).T
    
posteriors_seeds_A, pars_true_seeds_A = [], []
for exp_id in exp_ids_A:
    pp, load_vars = load_results_byname(exp_id)
    posteriors_seeds_A.append(pp)
    pars_true_seeds_A.append(load_vars['pars_true'])
Pmat_A = np.array([[-pp.eval(pt.reshape(1, -1), log=True)[0] for pp in P] for P, pt in zip (posteriors_seeds_A, pars_true_seeds_A)]).T

In [ ]:
nsim = (np.arange(run_opts['n_rounds']) + 1.0) * run_opts['n_train']
fig_nlogptrue = plt.figure()


sem = Pmat.std(axis=1) / np.sqrt(Pmat.shape[1])
meanlog = Pmat.mean(axis=1)
sem_A = Pmat_A.std(axis=1) / np.sqrt(Pmat_A.shape[1])
meanlog_A = Pmat_A.mean(axis=1)

plt.fill_between(nsim, meanlog_A-sem_A, meanlog_A+sem_A, color='c', alpha=0.3)
plt.fill_between(nsim, meanlog-sem, meanlog+sem, color='k', alpha=0.3)
h = plt.plot(nsim, Pmat.mean(axis=1),'k.-', label='APT')
hA = plt.plot(nsim, Pmat.mean(axis=1),'c>-', label='SNPE-A')

plt.ylabel('-log p(true parameters)')
plt.xlabel('Number of simulations (log scale)')
plt.legend(loc='best')